In [17]:
import os, re, time, json
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Dense, Input, Flatten, MaxPooling2D, BatchNormalization, Dropout, GlobalAveragePooling2D
from tensorflow import keras
import matplotlib.pyplot as plt

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
#Load dataset
(train_image, train_label), (test_image, test_label) = tf.keras.datasets.cifar10.load_data()
print('Dataset: ',(train_image.shape, train_label.shape), (test_image.shape, test_label.shape))

Dataset:  ((50000, 32, 32, 3), (50000, 1)) ((10000, 32, 32, 3), (10000, 1))


In [3]:
#Preprocess the dataset
train_image = train_image.astype('float32') / 255.0
test_image = test_image.astype('float32') / 255.0

In [18]:
resnet50 = ResNet50(weights = 'imagenet', classes = 10, include_top = False, input_shape = (32, 32, 3))


In [21]:
for layer in resnet50.layers:
    layer.trainable = True

x = resnet50.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation = 'relu')(x)
output = Dense(10, activation = 'softmax')(x)

# Create model
model = Model(inputs = resnet50.input, outputs = output)

In [22]:
model.summary()

Model: "functional_34"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 38, 38, 3) │          0 │ input_layer_2[0]… │
│ (ZeroPadding2D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 16, 16,    │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 16, 16,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 16, 16,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 18, 18,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 8, 8, 64)  │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 8, 8, 64)  │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 8, 8, 64)  │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 8, 8, 64)  │          0 │ conv2_block1_1_b… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 8, 8, 64)  │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 8, 8, 64)  │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 8, 8, 64)  │          0 │ conv2_block1_2_b… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 8, 8, 256) │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 8, 8, 256) │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 8, 8, 256) │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 8, 8, 256) │      1,024 │ conv2_block1_3_c

 Total params: 25,696,138 (98.02 MB)

 Trainable params: 25,643,018 (97.82 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [23]:
model.compile(optimizer = 'adam',
                loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                metrics = ['accuracy'])

# Train model
model.fit(train_image, train_label, validation_split = 0.1, shuffle = True, batch_size = 64, epochs = 10)


Epoch 1/15


c:\Users\sharh\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\backend\tensorflow\nn.py:599: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


704/704 ━━━━━━━━━━━━━━━━━━━━ 284s 378ms/step - accuracy: 0.5274 - loss: 1.5633 - val_accuracy: 0.1556 - val_loss: 3.6223
Epoch 2/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 296s 420ms/step - accuracy: 0.7425 - loss: 0.7808 - val_accuracy: 0.6394 - val_loss: 1.1312
Epoch 3/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 261s 370ms/step - accuracy: 0.7810 - loss: 0.6700 - val_accuracy: 0.5868 - val_loss: 1.3696
Epoch 4/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 239s 340ms/step - accuracy: 0.8034 - loss: 0.5922 - val_accuracy: 0.5946 - val_loss: 1.3112
Epoch 5/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 248s 353ms/step - accuracy: 0.8339 - loss: 0.4994 - val_accuracy: 0.7154 - val_loss: 0.8809
Epoch 6/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 244s 347ms/step - accuracy: 0.8525 - loss: 0.4460 - val_accuracy: 0.7058 - val_loss: 0.9345
Epoch 7/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 247s 351ms/step - accuracy: 0.8597 - loss: 0.4276 - val_accuracy: 0.7254 - val_loss: 1.0168
Epoch 8/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 247s 350ms/step - accuracy: 0.8569 - loss: 0.43

In [24]:

model.evaluate(test_image, test_label, batch_size = 64)

157/157 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - accuracy: 0.7929 - loss: 0.7981


[0.7997382879257202, 0.7930999994277954]